<a href="https://colab.research.google.com/github/ddPn08/kohya-sd-script-colab/blob/main/kohya_sd_script_lora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! nvidia-smi
! nvcc -V
! free -h

In [ ]:
# @markdown # Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

data_dir = "/content/data" # @param {type:"string"}
gdrive_data_dir = "/content/drive/MyDrive/AI/kohya/lora" # @param {type:"string"}
! ln -s {gdrive_data_dir} {data_dir}

In [ ]:
# @markdown # Initialize environment

! git clone https://github.com/kohya-ss/sd-scripts.git

import os

conda_bin = "/miniconda/bin/conda"
env_name = "kohya"
init_conda = f"""
eval "$({conda_bin} shell.bash hook)"
conda activate {env_name}
"""

if not os.path.exists(conda_bin):
    ! curl -O https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
    ! chmod +x Miniconda3-latest-Linux-x86_64.sh
    ! bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /miniconda
    ! rm Miniconda3-latest-Linux-x86_64.sh

def run_script(s):
    ! {s}

run_script(f"""
eval "$({conda_bin} shell.bash hook)"
cd sd-scripts
conda create -y -n {env_name} python=3.10 pip=22.2.2 cudatoolkit=11.3
conda activate {env_name}
conda install -y xformers -c xformers/label/dev
python -m pip install -q -r requirements.txt
python -m pip install -q tensorrt triton
python -m pip install -q torch==1.12.1+cu113 torchvision==0.13.1+cu113 --extra-index-url https://download.pytorch.org/whl/cu113
""")

os.environ[
    "LD_LIBRARY_PATH"
] = f"{os.environ['LD_LIBRARY_PATH']}:/usr/local/envs/{env_name}/lib"

In [ ]:
# @markdown # Settings

pretrained_model_name_or_path = "/content/drive/MyDrive/AI/models/Stable-diffusion/wd-1-4-anime_e1.ckpt"  # @param {type:"string"}
pretrained_model_is_v2 = True  # @param {type:"boolean"}
# @markdown ----

pretrained_model_resolution = "512,512"  # @param ["512,512", "768,768"] {allow-input: true}
input_dir = "/content/drive/MyDrive/AI/my-dataset/train" #@param {type:"string"}
reg_dir = "" #@param {type:"string"}
# @markdown ----

model_name = "my-model-v1"  # @param {type:"string"}
model_dir = f"{data_dir}/models/{model_name}"
model_out_dir = f"{model_dir}/result"
model_log_dir = f"{model_dir}/logs"

import os

os.makedirs(model_out_dir, exist_ok=True)
os.makedirs(model_log_dir, exist_ok=True)


In [ ]:
# @markdown # Optional | Preprocessing captions and tag information

preprocess_caption = False  # @param {type:"boolean"}
preprocess_tags = False  # @param {type:"boolean"}

meta_exists = os.path.exists("/content/meta.json")

if preprocess_caption:
    run_script(f"""{init_conda}
python sd-scripts/finetune/merge_captions_to_metadata.py {input_dir} meta.json \
{"--in_json=meta.json" if meta_exists else ""}
""")

meta_exists = os.path.exists("/content/meta.json")

if preprocess_tags:
    run_script(f"""{init_conda}
python sd-scripts/finetune/merge_dd_tags_to_metadata.py {input_dir} meta.json \
{"--in_json=meta.json" if meta_exists else ""}
""")

# @markdown ----

# @markdown # Optional | Advance acquisition of latents

prepare_buckets_latents = False  # @param {type:"boolean"}
batch_size = 4  # @param {type:"number"}
mixed_precision = "no" # @param ["no", "fp16", "bf16"]

run_script(f"""{init_conda}
python sd-scripts/finetune/prepare_buckets_latents.py \
    {input_dir} meta.json meta-lat.json {pretrained_model_name_or_path} \
    --batch_size={batch_size} \
    --max_resolution="{pretrained_model_resolution}" \
    --mixed_precision={mixed_precision} \
    {"--v2" if pretrained_model_is_v2 else ""}
""")

In [ ]:
# @markdown # Training
import os

meta_exists = os.path.exists("/content/meta-lat.json")

prior_loss_weight = 1.0  # @param{type:"number"}
batch_size = 1  # @param {type:"integer"}
learning_rate = 1e-3  # @param {type:"number"}
max_steps = 5000  # @param {type:"integer"}
mixed_precision = "no"  # @param ["no", "fp16", "bf16"]
gradient_accumulation_steps = 1  # @param {type:"integer"}
save_every_n_epochs = 5  # @param {type:"integer"}
model_ext = "safetensors" #@param ["diffusrs", "ckpt", "safetensors"]
network_weights = ""  # @param {type:"string"}
gradient_checkpointing = False  # @param{type:"boolean"}
use_8bit_adam = True  # @param{type:"boolean"}
enable_bucket = True  # @param{type:"boolean"}
color_aug = True  # @param{type:"boolean"}

additional_arguments = ""  # @param{type:"string"}

run_script(f"""{init_conda}
accelerate launch --num_cpu_threads_per_process 2 sd-scripts/train_network.py \
    --pretrained_model_name_or_path={pretrained_model_name_or_path} \
    --train_data_dir={input_dir} \
    {"--reg_data_dir=" + reg_dir if reg_dir else ""} \
    --output_dir={model_log_dir} \
    --prior_loss_weight={prior_loss_weight} \
    --resolution={pretrained_model_resolution} \
    --train_batch_size={batch_size} \
    --learning_rate={learning_rate} \
    --max_train_steps={max_steps} \
    --mixed_precision={mixed_precision} \
    --gradient_accumulation_steps={gradient_accumulation_steps} \
    --save_every_n_epochs={save_every_n_epochs} \
    --save_model_as={model_ext} \
    --network_module=networks.lora \
    --network_train_unet_only \
    --xformers \
    {"--use_8bit_adam" if use_8bit_adam else ""} \
    {"--enable_bucket" if enable_bucket else ""} \
    {"--color_aug" if color_aug else ""} \
    {"--gradient_checkpointing" if gradient_checkpointing else ""} \
    {"--v2" if pretrained_model_is_v2 else ""} \
    {"--in_json=/content/meta-lat.json" if meta_exists else ""} \
    {"--network_weights=" + network_weights if network_weights else ""} \
    {additional_arguments}

mv {model_log_dir}/last{"." + model_ext if model_ext != "diffusers" else ""} {model_out_dir}/
""")

In [ ]:
# @markdown # Generate images

prompt = "masterpiece, best quality, high quality, 1girl"  # @param {type:"string"}
uc = "worst quality, low quality, lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, jpeg artifacts, signature, watermark, username, blurry"  # @param {type:"string"}
lora_path = ""  # @param {type:"string"}
seed = -1  # @param {type:"integer"}
steps = 28  # @param {type:"integer"}
sampler = "euler_a"  # @param ['ddim', 'pndm', 'lms', 'euler', 'euler_a', 'heun', 'dpm_2', 'dpm_2_a', 'dpmsolver', 'dpmsolver++', 'dpmsingle', 'k_lms', 'k_euler', 'k_euler_a', 'k_dpm_2', 'k_dpm_2_a']
scale = 7.5  # @param {type:"number"}
height = 512 # @param {type:"slider", min:64, max:1024, step:64}
width = 512 # @param {type:"slider", min:64, max:1024, step:64}
batch_size = 1  # @param {type:"slider", min:1, max:64, step:1}
batch_count = 1  # @param {type:"slider", min:1, max:64, step:1}

run_script(f"""{init_conda}
accelerate launch --num_cpu_threads_per_process 2 sd-scripts/gen_img_diffusers.py \
    --ckpt={pretrained_model_name_or_path} \
    {"--v2" if pretrained_model_is_v2 else ""} \
    --outdir={data_dir}/outputs \
    --xformers \
    --network_module=networks.lora \
    --network_weights={lora_path} \
    --prompt="{prompt} --n {uc}" \
    {"--seed=" + seed if seed != -1 else ""} \
    --steps={steps} \
    --sampler={sampler} \
    --scale={scale} \
    --H={height} \
    --W={width} \
    --batch_size={batch_size} \
    --n_iter={batch_count}
""")


In [ ]:
# @markdown # Merge lora with checkpoint

import os

lora_model_path = ""  # @param {type:"string"}
save_to = ""  # @param {type:"string"}
ratios = 1  # @param {type:"number"}
precision = "bf16"  # @param ["float", "fp16", "bf16"]
save_precision = "auto"  # @param ["auto", "float", "fp16", "bf16"]
out_checkpoint_file = save_to or model_out_dir + "/checkpoints/merge." + model_ext

run_script(f"""{init_conda}
python sd-scripts/networks/merge_lora.py \
    --sd_model={pretrained_model_name_or_path} \
    --models={lora_model_path or model_out_dir + "/last." + model_ext} \
    --save_to={out_checkpoint_file} \
    --ratios={ratios} \
    --precision={precision} \
    --save_precision={precision if save_precision == "auto" else save_precision}
    {"--v2" if pretrained_model_is_v2 else ""}
""")

In [ ]:
# @markdown # Convert ckpt (or safetensors) to diffusers

import os

checkpoint = ""  # @param {type:"string"}
reference_model = "hakurei/waifu-diffusion"  # @param {type:"string"}
save_to = ""  # @param {type:"string"}
out_diffusers_dir = save_to or model_out_dir + "/diffusers/last"

run_script(f"""{init_conda}
python sd-scripts/tools/convert_diffusers20_original_sd.py \
    {checkpoint or out_checkpoint_file} \
    {out_diffusers_dir} \
    --reference_model={reference_model} \
    {"--v2" if pretrained_model_is_v2 else ""}
""")